<a href="https://colab.research.google.com/github/Alf-caput/P4_AlgoritmoGenetico/blob/dev3/P4_AlgGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRÁCTICA 4: Algoritmo genético

###Ana Robledano, Miguel Egido, Alfredo Robledano.

1.- Encontrar los máximos de la función "picos"

$$z = f(x,y) = 3(1-x)^2e^{-(x^2)-(y+1)^2}-10(x/5-x^3-y^5)e^{-x^2-y^2}-1/3e^{-(x+1)^2-y^2}$$

En el dominio $$\{x,y\} \in [-100,100]$$


2.- Dada la [Función de Rastringin](https://en.wikipedia.org/wiki/Rastrigin_function#:~:text=In%20mathematical%20optimization%2C%20the%20Rastrigin,has%20been%20generalized%20by%20Rudolph.), con A = 10 y  $$\{xi\} \in [-5.12,5.12]$$

Implementa dicha función en Python y minimízala para n=10

Compara el funcionamiento del Algoritmo Genético para cruce con un punto con cruce con dos puntos

### Instalación de librerías para resolver Algoritmos Genéticos con Python

In [1]:
!pip install deap
!pip install matplotlib
!pip install numpy
!pip install scipy
!pip install scoop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scoop: filename=scoop-0.7.2.0-py3-none-any.whl size=78113 sha256=507d5762f3b32901588b06743c631890d758d26df58c293f3a0f51bea01cf209
  Stored in directory: /root/.cache/pip/wheels/26/c6/da/088e4bffcfbc33fa40644ca636267bc801c8fd9eef973483db
Successfully built scoop


### Definición del problema ###


En este problema, la cadena cromosomática estaría formada por dos genes (cromosomas), que, al confinarse en forma de lista, quedaría como: $[x_i, y_i]$ con $i = 1, 2, ... , n$

La **población** del algoritmo **no cambia de tamaño** a lo largo de las generaciones, con lo que **n** será **constante**

In [ ]:
import random
import math
import numpy as np
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import matplotlib.pyplot as plt

#### NOTA SOBRE NÚMEROS ALEATORIOS:

Queremos generar números aleatorios entre -100 y 100 (sin incluir a estos). Para ello usaremos el módulo nativo random de generación de números pseudoaleatorios [Documentación librería random](https://docs.python.org/3/library/random.html) y `numpy.random`(para generación de vectores psudoaleatorios) [Documentación numpy.random](https://docs.scipy.org/doc/numpy-1.16.1/reference/routines.random.html). Aunque los números totalmente aleatorios no existen en programación.

Utilizamos, además, el método `uniform`: [Documentación método uniform](https://docs.python.org/3.6/library/random.html)

In [ ]:
# Importamos la librería
import random
# Fijamos la semilla. Si no cambiamos la semilla,
# la función generará los mismos número aleatorios
random.seed(0)
# Generamos los números pseudo-aleatorios en 2 list comprenhension
# para encapsular todos los datos en las listas x e y
x = [random.uniform(-100, 100) for i in range(10)]
y = [random.uniform(-100, 100) for i in range(10)]
print('x= ', x)
print('y= ', y)

Otra posibilidad es generar dos vectores de diez valores comprendidos entre -100 y 100, con una forma (1,10) (1 fila y 10 columnas). Serían dos vectores de tipo fila con diez valores

In [ ]:
import numpy as np
# Fijamos la semilla
np.random.seed(0)
# Generamos los números pseudo-aleatorios
# Son los mismos mientras no cambiemos la semilla
# Numpy genera directamente los vectores
z = np.random.uniform(-100, 100, 10).reshape(1, 10)
t = np.random.uniform(-100, 100, 10).reshape(1, 10)
print('z= ', z)
print('t= ', t)

Hay que tener en cuenta que las soluciones deben ser válidas y una solución sería **no válida** si alguna de las variables independientes se saliera de los **rangos establecidos**. Es común en los problemas de optimización tener restricciones en las variables, por lo que siempre tendremos que **comprobar la validez de nuestras soluciones**

#### NOTA 2: ESPACIO DE BÚSQUEDA O DOMINIO DE LA SOLUCIÓN -

Vamos a resolverlo con Algoritmo Genético como ejercício, aunque cualquier algoritmo de optimización basado en gradiente de los que vienen incluidos en el módulo `optimize` de `scipy`([módulo optimize en scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize)) nos valdría para obtener una solución a nuestro problema de manera sencilla, ya que nuestra función es **convexa**

Recordar que: Los AGs se deben emplear en **problemas de optimización** en los que la solución óptima **no se conozca** y ni siquiera tengamos una **aproximación** a la misma o incluso **ni siquiera se tenga la certeza** de que podamos obtenerla.

Aunque el **uso de AGs no nos da certeza de obtener la solución óptima**, si obtendremos una solución **bastante buena** y en un **tiempo razonable**. Siempre vamos a terminar con una solución al problema que **será mejor que realizar una búsqueda totalmente aleatoria**

### Definición de la función del enunciado ###

In [ ]:
# Importamos las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
# Definimos la función objetivo
import math
f = lambda x, y: 3*(1-x)**2*math.exp(-(x**2)-(y+1)**2)-10(x/5-x**3-y**5)*math.exp(-x**2-y**2)-1/3*math.exp(-x+1)**2-y**2

[numpy.linspace](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html)

In [ ]:
# Generamos vectores que barran todo el dominio
x = np.linspace(-100, 100, 1000)
y = np.linspace(-100, 100, 1000)
print('x= ', x)
print('y= ', y)

[numpy.meshgrid](https://numpy.org/doc/stable/reference/generated/numpy.meshgrid.html)

[numpy.ravel](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html)

[numpy.c_](https://numpy.org/doc/stable/reference/generated/numpy.c_.html)

In [ ]:
# Evaluamos todos los valores de las variables


[matplotlib.pyplot.gca](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.gca.html)

In [ ]:
# Representamos los resultados obtenidos

A partir de ahora veremos los siguientes pasos:
1. Definición del problema y generación de la población inicial
- 1.1 Creación del problema
- 1.2 Creación de la plantilla del individuo
- 1.3 Crear individuos aleatorios y población inicial
2. Función objetivo y operadores genéticos
- 2.1 Función objetivo
3. Incorporación de los operadores genéticos
4. Últimos pasos: Algortimo Genético como caja negra
- 4.1 Configuración algoritmo genético
- 4.2 Resultados del algoritmo genético
5. ¿Cómo conseguir resultados consistentes?
6. Convergencia del algoritmo